In [7]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from numpy.random import default_rng
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

# This script generates data simulating history of shipments to a warehouse. 
# The simulation is based on the article: 
#   Michael Lingzhi L, Elliott Wolf, Daniel Wintz (2020): Duration-of-stay storage assignment
#   under uncertainty. Published as a conference paper at ICLR 2020. http://arXiv:1903.05063v3 [cs.LG] 1Feb 2020
# The script is organized as follows:
# 1. _prodt_ table is formed containing categorical information about the shipment - 
#    supplier, customer, outbound location, customer type, product group, product packaging, 
#    nutritional value, content, expiration period, reference to the trend and seasonality function (TSF) of demand
# 2. In while cycle from start date to end date of the history, fact data of deliveries are calculated. The fact data are date, 
#    volume (number of pallets), total storage time/duration, number of pallets unsold until the expiration date, 
#    threshold value of the duration, percentiles of cumulative duration of storage (DoS) distribution for pallets. 
#    Each iteration of the loop goes through _prodt_ and the result is added to the dfData set.
# 3. The fields supplier, product group, packaging, nutritional value, product content are combined 
#    into a text description. 3 different orders are used to combine field values. The order of the fields 
#    is chosen randomly. The resulting descriptions are stored in field dfData.txt. 
# 4. Reference tables for customers, outbound locations, suppliers, product groups are formed and saved
# 5. dfData is saved in the inidata.csv file

WORKING_DIRECTORY = 'C:/Pilot/test/'
os.chdir (WORKING_DIRECTORY)

fpLog = Path ('log.txt')

with open (fpLog, 'w') as flog:
    print ('Start data simulation at : ', datetime.now(), file = flog)

# intersection of supplier and nutritional value
# 'NA' means none
supnt = pd.DataFrame(
    {
        'SUP01' : ['FATFR','LCTFR','SGRFR','DIET','NA',np.nan,np.nan],
        'SUP02' : ['ORG','PRVFR',np.nan,np.nan,np.nan,np.nan,np.nan],
        'SUP03' : ['FATFR','LCTFR','SGRFR','DIET','PRVFR','NA',np.nan],
        'SUP04' : ['FATFR','LCTFR','SGRFR','DIET','PRVFR','ORG','NA'],
        'SUP05 CHEES' : ['ORG','NA',np.nan,np.nan,np.nan,np.nan,np.nan],
        'SUP06 ICRM' : ['NA',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
    }
)
# intersection of supplier and product group
suppr = pd.DataFrame(
    {
        'SUP01' : ['MLK','CURD','ICRM','KFR','FBM','CHEES','CHEES SLC','SCRM','JGRT','PIZZA'],
        'SUP02' : ['MLK','CURD','ICRM','KFR','FBM','CHEES','CHEES SLC','SCRM','JGRT','PIZZA'],
        'SUP03' : ['MLK','CURD','KFR','FBM','SCRM',np.nan,np.nan,np.nan,np.nan,np.nan],
        'SUP04' : ['MLK','CURD','JGRT','ICRM',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
        'SUP05 CHEES' : ['CHEES','CHEES SLC','PIZZA',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
        'SUP06 ICRM' : ['ICRM',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
    }
)
# intersection of product group and volume
prvol = pd.DataFrame(
    {
        'MLK': ['100ml','300ml','500ml','1L','2L','5L'],
        'CURD': ['100g','300g','500g','1kg','NA',np.nan],
        'ICRM': ['100g','300g','500g','1kg',np.nan,np.nan],
        'KFR': ['100ml','300ml','500ml','1L',np.nan,np.nan],
        'FBM': ['100ml','300ml','500ml','1L',np.nan,np.nan],
        'CHEES': ['100g','300g','500g','1kg','HEAD',np.nan],
        'CHEES SLC': ['100g',np.nan,np.nan,np.nan,np.nan,np.nan],
        'SCRM': ['100ml','300ml',np.nan,np.nan,np.nan,np.nan],
        'JGRT': ['100ml','300ml','500ml','1L','2L',np.nan],
        'PIZZA': ['25cm','30cm',np.nan,np.nan,np.nan,np.nan]
    }
)
# intersection of product group and packaging
prpak = pd.DataFrame(
    {
        'MLK': ['PBTL','GBTL','PCNR','TPAK'],
        'CURD': ['PGLS','PCNT',np.nan,np.nan],
        'ICRM': ['PRCNT',np.nan,np.nan,np.nan],
        'KFR': ['PBTL','GBTL','TPAK',np.nan],
        'FBM': ['PBTL','GBTL','TPAK',np.nan],
        'CHEES': ['PCNT','VACUM',np.nan,np.nan],
        'CHEES SLC': ['PCNT',np.nan,np.nan,np.nan],
        'SCRM': ['PGLS',np.nan,np.nan,np.nan],
        'JGRT': ['PCNT',np.nan,np.nan,np.nan],
        'PIZZA': ['VACUM','PRCNT',np.nan,np.nan]
    }
)
# intersection of volume and packaging
vlpak = pd.DataFrame(
    {
        '100ml': ['PGLS','PCNT',np.nan,np.nan,np.nan],
        '300ml': ['PGLS','PCNT','PBTL',np.nan,np.nan],
        '500ml': ['PGLS','PCNT','PBTL','GBTL',np.nan],
        '1L': ['PCNT','PBTL','GBTL','PCNR','TPAK'],
        '2L': ['PBTL','PCNR',np.nan,np.nan,np.nan],
        '5L': ['PCNR',np.nan,np.nan,np.nan,np.nan],
        '100g': ['PCNT','VACUM','PRCNT',np.nan,np.nan],
        '300g': ['PCNT','VACUM','PRCNT',np.nan,np.nan],
        '500g': ['PCNT','VACUM','PRCNT',np.nan,np.nan],
        '1kg': ['PCNT','VACUM','PRCNT',np.nan,np.nan],
        'HEAD': ['PCNT','VACUM','PRCNT',np.nan,np.nan],
        '25cm': ['PRCNT','VACUM',np.nan,np.nan,np.nan],
        '30cm': ['PRCNT','VACUM',np.nan,np.nan,np.nan],    
    }
)
# intersection of product group and content
prcnt = pd.DataFrame(
    {
        'MLK': ['1.0%','1.5%','3.2%','3.6%','5%'],
        'CURD': ['1.0%','5%','9%',np.nan,np.nan],
        'ICRM': ['CREM','FI','NA',np.nan,np.nan],
        'KFR': ['0.5%','1.5%','2.5%','5%',np.nan],
        'FBM': ['3.2%','4%','6%',np.nan,np.nan],
        'CHEES': ['25%','45%','50%','60%',np.nan],
        'CHEES SLC': ['25%','45%','50%','60%',np.nan],
        'SCRM': ['10%','15%','20%','25%','30%'],
        'JGRT': ['1.0%','5%','9%',np.nan,np.nan],
        'PIZZA': ['TCK','TIN',np.nan,np.nan,np.nan]
    }
)
# intersection of product group and nutrition value
prntr = pd.DataFrame(
    {
        'MLK': ['FATFR','LCTFR','NA',np.nan,np.nan],
        'CURD': ['FATFR','NA',np.nan,np.nan,np.nan],
        'ICRM': ['LCTFR','DIET','NA',np.nan,np.nan],
        'KFR': ['DIET','NA',np.nan,np.nan,np.nan],
        'FBM': ['DIET','NA',np.nan,np.nan,np.nan],
        'CHEES': ['ORG','LWFAT','NA',np.nan,np.nan],
        'CHEES SLC': ['ORG','LWFAT','NA',np.nan,np.nan],
        'SCRM': ['ORG','NA',np.nan,np.nan,np.nan],
        'JGRT': ['PRVFR','NA',np.nan,np.nan,np.nan],
        'PIZZA': ['VEGN','DIET','NA',np.nan,np.nan]
    }
)
# intersection of content and nutrition value
cnval = pd.DataFrame(
    {
        '0.5%': ['FATFR','NA',np.nan,np.nan,np.nan,np.nan],
        '1.0%': ['FATFR','NA',np.nan,np.nan,np.nan,np.nan],
        '1.5%': ['FATFR','NA',np.nan,np.nan,np.nan,np.nan],
        '2.5%': ['DIET','ORG','PRVFR','LCTFR','NA',np.nan],
        '3.2%': ['DIET','ORG','PRVFR','LCTFR','NA',np.nan], 
        '3.6%': ['DIET','ORG','PRVFR','LCTFR','NA',np.nan],
        '5%'  : ['NA',np.nan,np.nan,np.nan,np.nan,np.nan], 
        '6%'  : ['NA',np.nan,np.nan,np.nan,np.nan,np.nan],
        '9%'  : ['ORG','PRVFR','LCTFR','NA',np.nan,np.nan],
        'CREM': ['LCTFR','NA',np.nan,np.nan,np.nan,np.nan],
        'FI'  : ['NA',np.nan,np.nan,np.nan,np.nan,np.nan],
        '10%' : ['NA',np.nan,np.nan,np.nan,np.nan,np.nan],
        '15%' : ['NA',np.nan,np.nan,np.nan,np.nan,np.nan],
        '20%' : ['PRVFR','NA',np.nan,np.nan,np.nan,np.nan],
        '30%' : ['ORG','NA',np.nan,np.nan,np.nan,np.nan],
        '25%' : ['ORG','NA',np.nan,np.nan,np.nan,np.nan],
        '45%' : ['PRVFR','NA',np.nan,np.nan,np.nan,np.nan],
        '50%' : ['PRVFR','NA',np.nan,np.nan,np.nan,np.nan],
        '60%' : ['ORG','NA',np.nan,np.nan,np.nan,np.nan],
        'TCK' : ['ORG','NA',np.nan,np.nan,np.nan,np.nan],
        'TIN' : ['PRVFR','NA',np.nan,np.nan,np.nan,np.nan],
        'NA'  : ['DIET','ORG','PRVFR','LCTFR','NA',np.nan]
    }
)

# Form dataframe _prodt_ which joins supplier, product group, volume, packing, content, nutrient value 
# Note: for brevity, in all scripts 'product' means 'product group'

suppr = suppr.melt (var_name ='supplier', value_name ='product_')
suppr = (suppr := suppr[suppr.product_.notna()]).rename (columns ={'product_': 'product'})
supnt = (dt := supnt.melt (var_name ='supplier', value_name ='nvalue'))[dt.nvalue.notna()]

suplr = suppr.merge (supnt, left_on ='supplier', right_on ='supplier')

vlpak = (dt := vlpak.melt (var_name ='volume',  value_name ='packing'))[dt.packing.notna()]
prpak = (dt := prpak.melt (var_name ='product', value_name ='packing'))[dt.packing.notna()]
prvol = (dt := prvol.melt (var_name ='product', value_name ='volume' ))[dt.volume.notna() ]
prcnt = (dt := prcnt.melt (var_name ='product', value_name ='content'))[dt.content.notna()]
prntr = (dt := prntr.melt (var_name ='product', value_name ='nvalue' ))[dt.nvalue.notna() ]
cnval = (dt := cnval.melt (var_name ='content', value_name ='nvalue' ))[dt.nvalue.notna() ]

prodt = prvol.merge (vlpak, on ='volume')
prodt = prodt.merge (prpak, how ='inner', on =('packing','product'))

prodt_ = prcnt.merge (cnval, on ='content')
prodt_ = prodt_.merge(prntr, how = 'inner', on =('nvalue','product'))

_prodt_ = prodt.merge(prodt_, on ='product')
_prodt_ = suplr.merge(_prodt_, how = 'inner', on =('nvalue','product'))

# Add product's expiration period ('term'). Expiration period is determined by product group, 
# product group and packing, product group and content, product group and nvalue. 
# Expiration period is one of the most important features for forecasting. Therefore 'term' is not included in
# input data. This info can only be obtained from descriptions. Expiration period is used 
# to calculate sales charts only. 

arex =[
            {'product': 'CURD',   'packing': np.nan, 'content': '1.0%', 'nvalue': np.nan, 'term': 25}, 
            {'product': 'CURD',   'packing': np.nan, 'content': '5%',   'nvalue': np.nan, 'term': 25}, 
            {'product': 'CURD',   'packing': np.nan, 'content': '9%',   'nvalue': np.nan, 'term': 15}, 
            {'product': 'CURD',   'packing': 'NA',   'content': np.nan, 'nvalue': np.nan, 'term': 5 }, 
            {'product': 'JGRT',   'packing': np.nan, 'content': '1.0%', 'nvalue': np.nan, 'term': 25}, 
            {'product': 'JGRT',   'packing': np.nan, 'content': '5%',   'nvalue': np.nan, 'term': 25}, 
            {'product': 'JGRT',   'packing': np.nan, 'content': '9%',   'nvalue': np.nan, 'term': 15}, 
            {'product': 'JGRT',   'packing': np.nan, 'content': np.nan, 'nvalue':'PRVFR', 'term': 5 }, 
            {'product': 'MLK',    'packing': 'GBTL', 'content': np.nan, 'nvalue': np.nan, 'term': 5 },
            {'product': 'MLK',    'packing': 'PBTL', 'content': np.nan, 'nvalue': np.nan, 'term': 20}, 
            {'product': 'MLK',    'packing': 'PCNR', 'content': np.nan, 'nvalue': np.nan, 'term': 20}, 
            {'product': 'MLK',    'packing': 'TPAK', 'content': np.nan, 'nvalue': np.nan, 'term': 60}, 
            {'product': 'SCRM',   'packing': np.nan, 'content': '10%',  'nvalue': np.nan, 'term': 15}, 
            {'product': 'SCRM',   'packing': np.nan, 'content': '15%',  'nvalue': np.nan, 'term': 15}, 
            {'product': 'SCRM',   'packing': np.nan, 'content': '20%',  'nvalue': np.nan, 'term': 10}, 
            {'product': 'SCRM',   'packing': np.nan, 'content': '25%',  'nvalue': np.nan, 'term': 10}, 
            {'product': 'SCRM',   'packing': np.nan, 'content': '30%',  'nvalue': np.nan, 'term': 10}, 
            {'product': 'ICRM',   'packing': np.nan, 'content': np.nan, 'nvalue': np.nan, 'term': 60}, 
            {'product': 'KFR',    'packing': 'GBTL', 'content': np.nan, 'nvalue': np.nan, 'term': 5 },
            {'product': 'KFR',    'packing': 'PBTL', 'content': np.nan, 'nvalue': np.nan, 'term': 20}, 
            {'product': 'KFR',    'packing': 'TPAK', 'content': np.nan, 'nvalue': np.nan, 'term': 60}, 
            {'product': 'FBM',    'packing': 'GBTL', 'content': np.nan, 'nvalue': np.nan, 'term': 5 },
            {'product': 'FBM',    'packing': 'PBTL', 'content': np.nan, 'nvalue': np.nan, 'term': 20}, 
            {'product': 'FBM',    'packing': 'TPAK', 'content': np.nan, 'nvalue': np.nan, 'term': 60}, 
            {'product': 'CHEES',   'packing': np.nan, 'content': np.nan, 'nvalue': np.nan, 'term': 30}, 
            {'product': 'CHEES SLC','packing': np.nan, 'content': np.nan, 'nvalue': np.nan, 'term': 20}, 
            {'product': 'PIZZA',  'packing': np.nan, 'content': np.nan, 'nvalue': np.nan, 'term': 120} 
        ]
ex = pd.DataFrame (arex)
exnul = ex[['packing','content','nvalue']].isnull() 
exnul['allnan'] = (exnul['packing'] & exnul['content'] & exnul['nvalue'])
exnul['cvnnan'] = ((~exnul['packing']) & exnul['content'] & exnul['nvalue'])
exnul['pvnnan'] = (exnul['packing'] & (~exnul['content']) & exnul['nvalue'])
exnul['pkcnan'] = (exnul['packing'] & exnul['content'] & (~exnul['nvalue']))
_prodt_ = _prodt_.merge(ex[exnul['allnan']][['product','term']], how = 'left', on=('product'))
_prodt_ = _prodt_.rename(columns={'term': "term0"})
_prodt_ = _prodt_.merge(ex[exnul['cvnnan']][['product','packing','term']], how = 'left', on=('product','packing'))
_prodt_ = _prodt_.rename(columns={'term': "term1"})
_prodt_ = _prodt_.merge(ex[exnul['pvnnan']][['product','content','term']], how = 'left', on=('product','content'))
_prodt_ = _prodt_.rename(columns={'term': "term2"})
_prodt_ = _prodt_.merge(ex[exnul['pkcnan']][['product','nvalue', 'term']], how = 'left', on=('product','nvalue' ))
_prodt_ = _prodt_.rename(columns={'term': "term3"})
_prodt_['term'] = _prodt_[['term0','term1','term2','term3']].min (axis = 1)
_prodt_ = _prodt_[['supplier', 'product', 'nvalue', 'volume', 'packing', 'content', 'term']]

_prodt_.sort_values (
    by = ['supplier', 'product', 'nvalue', 'volume', 'packing', 'content', 'term'], 
    ignore_index = True
    ).to_html (Path ('ref_descriptions.html'))

# Form reference table of descriptions. I use 'did' as input feature for forecasting models 
# as if we received and structured all the information from the descriptions without errors.
# 'Did' is not included in input data for product identification. 

dfdesc = _prodt_.loc (axis = 1)[['supplier', 'product', 'nvalue', 'volume', 'packing', 'content']]
dfdesc['did'] = np.arange (1, dfdesc.shape[0] + 1, dtype = int)

# Create reference table prlt which assigns TSFs to product groups. 

prlt = pd.DataFrame(
    {
        'product': ['MLK','CURD','ICRM','KFR','FBM','CHEES','CHEES SLC','SCRM','JGRT','PIZZA'],
#        'lt'     : [    0,     1,    4,    2,    2,    3,        3,     1,     0,     3]
        'lt'     : [    0,     1,    1,    1,    0,    0,         1,     1,     0,     1]
    }
)

_prodt_ = _prodt_.merge (prlt, how = 'left', on=('product'))

HLENGTH  = 6    # duration of the history is given by number of years 
INFINITY = 13   # auxiliary arrays are calculated for a duration greater than HLENGTH with a margin 
                # INFINITY is given by the number of years as well

dstart = date.fromisoformat ('2019-01-01')                      # warehouse start date
dstop  = dstart + relativedelta (years = HLENGTH, days = -1)    # end date of history
d64start = np.datetime64('2019-01-01', 'D')                     # warehouse start date in datetime64 format/type
ihdays = (dstop - dstart).days + 1                              # duration of the history in days

# Define TSFs

# In the code commented bellow function fnTrend2 calculates TSF with pooly conditioned year seasonality.
# TSF lags half a period every odd year. 
# Then 2 variants of such TSF are formed with period of 4 and 2 months

def fnTrend2 (ipmonths, imaxpoint, k1, b):

# ipmonths is period of TSF and has to contain integer number of months. 
# For example, 3 means that demand repeats every 3 months 
# imaxpoint contains the coordinate of TSF's maximum point for even periods. 
# imaxpoint is specified in days from the beginning of the period.
# k determines a slope of TSF, b determines a value at the minimum point

    lY = []

    for i in range (INFINITY*12//ipmonths):
        dbm = dstart + relativedelta (months = i*ipmonths)
        dem = dstart + relativedelta (months = (i+1)*ipmonths, days = -1)
        ipdays = (dem - dbm).days + 1
        nax = np.arange (ipdays)
        nay = np.arange (ipdays, dtype = np.float32)
        sls1  = slice (0, imaxpoint + 1)
        sls2x = slice (0, ipdays - imaxpoint - 1)
        sls2y = slice (imaxpoint + 1, ipdays)
        nay[sls1] = nax[sls1]*k1
        k2 = nay[imaxpoint]/(ipdays - imaxpoint - 1)
        nay[sls2y] = nay[imaxpoint] - (nax[sls2x] + 1)*k2
        lY = lY + (nay + b).tolist()
    
    naY1 = np.asarray (lY)
    naY2 = np.roll (naY1, ipdays//2)
    naX = np.arange (dstart, dstart + relativedelta (days = naY1.shape[0]), timedelta (days = 1), dtype = date) 
    naY = np.asarray([naY1[i] if naX[i].year%2 == 0 else naY2[i] for i in range(naY1.shape[0])])

    return naY#, naY1, naY2, naX
'''
MAXPOINT = 60  #number of days
PERIOD   = 4   #number of months
k1 = 0.6
b  = 5

btrend00 = fnTrend2 (PERIOD, MAXPOINT, k1, b)
#naY, naY1, naY2, naX = fnTrend (PERIOD, MAXPOINT, k1, b)

#pd.DataFrame (naY[0:2199], columns = ['TSF1']).to_csv (Path ('picture01.csv'), index = False)
#pd.DataFrame (naY1[0:2199], columns = ['TSF2']).to_csv (Path ('picture02.csv'), index = False)

MAXPOINT = 30  #number of days
PERIOD   = 2   #number of months
k1 = 1.2
b  = 5.0

btrend01 = fnTrend2 (PERIOD, MAXPOINT, k1, b)
'''

# fnTrend1 calculates TSF with well conditioned year seasonality.
# 2 variants of such TSF are formed with period 4 and 2 months

def fnTrend1 (ipmonths, imaxpoint, k1, b):

# ipmonths is period of TSF and has to contain integer number of months. 
# For example, 3 means that demand repeats every 3 months 
# imaxpoint contains the coordinate of TSF's maximum point. 
# imaxpoint is specified in days from the beginning of the period.
# k determines a slope of TSF, b determines a value at the minimum point

    lY = []

    for i in range (INFINITY*12//ipmonths):
        dbm = dstart + relativedelta (months = i*ipmonths)
        dem = dstart + relativedelta (months = (i+1)*ipmonths, days = -1)
        ipdays = (dem - dbm).days + 1
        nax = np.arange (ipdays)
        nay = np.arange (ipdays, dtype = np.float32)
        sls1  = slice (0, imaxpoint + 1)
        sls2x = slice (0, ipdays - imaxpoint - 1)
        sls2y = slice (imaxpoint + 1, ipdays)
        nay[sls1] = nax[sls1]*k1
        k2 = nay[imaxpoint]/(ipdays - imaxpoint - 1)
        nay[sls2y] = nay[imaxpoint] - (nax[sls2x] + 1)*k2
        lY = lY + (nay + b).tolist()

    return np.asarray (lY)

MAXPOINT = 60  #number of days
PERIOD   = 4   #number of months
k1 = 0.6
b  = 5

btrend00 = fnTrend1 (PERIOD, MAXPOINT, k1, b)

MAXPOINT = 30  #number of days
PERIOD   = 2   #number of months
k1 = 1.2
b  = 5.0

btrend01 = fnTrend1 (PERIOD, MAXPOINT, k1, b)

# fnWave calculates smooth TSFs which are not used in the current version of 
# this script (see reference table prlt) 

def fnWave (ipmonths, ampli, b):

    lY = []

    for i in range (INFINITY*12//ipmonths):
        dbm = dstart + relativedelta (months = i*ipmonths)
        dem = dstart + relativedelta (months = (i+1)*ipmonths, days = -1)
        ipdays = (dem - dbm).days + 1
        nax = np.arange (ipdays)
        nay = np.arange (ipdays, dtype = np.float32)
        nay = (np.sin (2*3.14*nax/ipdays) + b)*ampli
        lY = lY + nay.tolist()

    return np.asarray (lY)

# Trend #2: Wave per month

PERIOD = 1      #number of months 
AMPLIFIER = 10
b = 1.5

btrend02 = fnWave (PERIOD, AMPLIFIER, b)

# Trend #3: Wave per quater

PERIOD = 3      #number of months 
AMPLIFIER = 20
b = 1.5

btrend03 = fnWave (PERIOD, AMPLIFIER, b)

# Trend #4: Wave per year

PERIOD = 12      #number of months 
AMPLIFIER = 40
b = 1.5

btrend04 = fnWave (PERIOD, AMPLIFIER, b)

trends = np.ndarray (shape = (5, btrend00.shape[0]))
trends[0][:] = btrend00
trends[1][:] = btrend01
trends[2][:] = btrend02
trends[3][:] = btrend03
trends[4][:] = btrend04

# Add clients/customers ('client'), client/customer types ('tid'), outbound locations (ol). 
# Inbound location (il) is equal to supplier. 
# Add the following reference tables: 
# clnt lists clients and their types
# ctpr bundles clients and product groups
# olil bundles outbound locations and suppliers
# clol bundles clients and outbound locations
# outl specifies a gain for each outbound location

# Client type has 3 values: restaurants - 1, hotels - 2, shops - 3. 
# Table ctpr (client type x product group) determines some restrictions:  
# - restaurants consume cheese, ice cream and milk,  
# - hotels pizza, ice cream, milk, cheese,  
# - stores take away all product groups. 

clnt = pd.DataFrame(
    {
        'client': ['CLN01','CLN02','CLN03','CLN04','CLN05'],
        'tid'   : [      1,      1,      2,      3,      3]
    }
)
ctpr = pd.DataFrame(
    {
        'CLN01' : ['MLK','ICRM','CHEES',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
        'CLN02' : ['MLK','ICRM','CHEES',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
        'CLN03' : ['MLK','ICRM','CHEES','PIZZA',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
        'CLN04' : ['MLK','CURD','ICRM','KFR','FBM','CHEES','CHEES SLC','SCRM','JGRT','PIZZA'],
        'CLN05' : ['MLK','CURD','ICRM','KFR','FBM','CHEES','CHEES SLC','SCRM','JGRT','PIZZA'],
    }
)
clol = pd.DataFrame(
    {
        'CLN01' : ['LO1',np.nan],
        'CLN02' : ['LO2',np.nan],
        'CLN03' : ['LO3',np.nan],
        'CLN04' : ['LO4', 'LO5'],
        'CLN05' : ['LO6', 'LO7'],
    }
)
olil = pd.DataFrame(
    {
        'LO1' : ['SUP01',       np.nan,      np.nan],
        'LO2' : ['SUP05 CHEES','SUP04',      np.nan],
        'LO3' : ['SUP05 CHEES','SUP03','SUP06 ICRM'],
        'LO4' : ['SUP01',       np.nan,      np.nan],
        'LO5' : ['SUP01',       np.nan,      np.nan],
        'LO6' : ['SUP02',       np.nan,      np.nan],
        'LO7' : ['SUP02',       np.nan,      np.nan]
    }
)
outl = pd.DataFrame(
    {
        'oloc': ['LO1','LO2','LO3','LO4','LO5','LO6','LO7'],
        'ampli':[  1.4,  1.6,  2.1,  4.8,  5.1,  5.4,  4.2]
    }
)

# Add client and outbound location to dataframe _prodt_   

ctpr = ctpr.melt (var_name='client', value_name='product_')
ctpr = ctpr[ctpr.product_.notna()].rename (columns={'product_': 'product'})

clol = clol.melt (var_name='client', value_name='oloc')
clol = clol[clol.oloc.notna()]

ctpr = ctpr.merge (clol, on='client')

olil = olil.melt (var_name='oloc', value_name='supplier')
olil = olil[olil.supplier.notna()]

olpr = olil.merge (suppr, how = 'inner', on='supplier')

olpr = olpr.merge (ctpr, how = 'inner', on=('product','oloc'))
olpr = olpr.merge (outl, how = 'inner', on='oloc')

_prodt_ = _prodt_.merge (olpr, how = 'inner', on =('supplier', 'product'))

# Prepare while-cycle to add shipments
# Sales charts are calculated for shipments in the array sales.  
# The diff array is needed to calculate DoS. On the cumulative sales chart, 
# I build a spline of the first order and calculate percentiles from the spline.  
# However, if there were no sales, then the same chart value corresponds to different days.  
# This results in an error when calculating percentiles. To get around the ambiguity, 
# minor additions to the cumulative graph are calculated in diff for the absence of sales.
# The array k contains coefficients that determine decline in the demand 
# curve for products of the calculated shipments.

rng = default_rng()

START_PERIOD = 30 # dates of first shipments are taken from interval 0 - START_PERIOD
MAXTERM = 120     # defines a maximum size of sales chart

sales0   = np.zeros ((_prodt_.shape[0], MAXTERM + 1))
save     = np.zeros ((_prodt_.shape[0], MAXTERM))
na_diff_ = np.zeros (sales0.shape)
k        = np.zeros (_prodt_.shape[0])

# _prodt_.nsl stores unsold part (overdue product) of shipment
# _prodt_.drn contains total sale time of shipment.
# _prodt_.qnt is volume of shipment.
# _prodt_.stop is threshold time/duration of shipment. If the total sale time of shipment 
# twice less than the threshold time volume of next shipment increses twice   

_prodt_['nsl'] = 0.0
_prodt_['drn'] = np.int32(0)
_prodt_['qnt'] = 0.0
_prodt_['stop']= np.int32(0)

lhash = ['supplier', 'product', 'nvalue', 'volume', 'packing', 'content', 'term', 'lt', 'client', 'oloc']
_prodt_['phash'] = _prodt_.loc(axis = 1)[lhash].apply (lambda x: hash(tuple(x)), axis = 1)

with open (fpLog, 'a') as flog:
    print ('TERM x LT = ', _prodt_.loc(axis=1)[['term','lt']].drop_duplicates(), file = flog)

with open (fpLog, 'a') as flog:
    print ('_prodt_.columns = ', _prodt_.columns, file = flog)
    print ('_prodt_.shape = ', _prodt_.shape, file = flog)
    lcatdesc = ['supplier', 'product', 'nvalue', 'volume', 'packing', 'content']
    print ('SUPPLIER x PRODUCT x NVALUE x VOLUME x PACKING x CONTENT = ', _prodt_.loc(axis=1)[lcatdesc].drop_duplicates().shape[0], file = flog)

IND = 432
HASHID = _prodt_.phash.loc(axis = 0)[IND]
lIND = _prodt_.index[_prodt_.phash == HASHID].tolist()
RNUMBER = _prodt_.index.get_loc (lIND[0]) if len(lIND) > 0 else -100

count = 0   # while-cycle counter

# Shipments' Calculation Cycle
# A new shipment is calculated for each row of _prodt_. 
# If a shipment date (see below) + the total sale time of the shipment is 
# greater than the end date of the history, then the line is removed from _prodt_
# The loop ends when the number of rows in prodt is 0.

while _prodt_.shape[0] > 0:

    with open(fpLog, 'a') as flog:
        print ('COUNT = ', count, file = flog)

    # Since the number of rows in _prodt_ can change, at the beginning of each iteration 
    # main arrays are resized to match the number of rows in _prodt_.

    sales0   = np.resize (sales0, (_prodt_.shape[0], MAXTERM + 1))
    save     = np.resize (sales0, (_prodt_.shape[0], MAXTERM))
    na_diff_ = np.resize (na_diff_, sales0.shape)
    k        = np.resize (k,       _prodt_.shape[0])
    sales0.fill(0), save.fill(0), na_diff_.fill(0), k.fill(0)

    # _prodt_.start is the shipment date. It is the number of days from the start of the warehouse. 
    # If the cycle counter is equal to zero, then the shipment date is randomly selected from 
    # the interval 0 - START_PERIOD. If the cycle counter is greater than zero, then 
    # the shipment date is calculated as shipment date of the previous shipment + the interval for 
    # which the previous shipment was sold (prodt.drn) + a random number of days from 1 to 4.

    if count == 0:
        _prodt_['start'] = rng.integers (low = 0, high = START_PERIOD, size = _prodt_.shape[0], dtype = np.int32, endpoint = True)
    else:
        nadelays = rng.integers (low = 1, high = 4, size = _prodt_.shape[0], dtype = np.int32, endpoint = True)
        _prodt_['start'] = _prodt_['start'] + _prodt_['drn'] + nadelays
    
    sales = sales0[:, 1:]
    sales += np.arange (MAXTERM)
    
    # Initialize auxiliary arrays for expiration periods and demand gains

    term   = _prodt_['term'].to_numpy()
    ampli  = _prodt_['ampli'].to_numpy()

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('term:', file = flog)
            print (term[RNUMBER], file = flog)
            print ('nsl:',  file = flog)
            print (_prodt_.nsl[RNUMBER], file = flog)
            print ('qnt:',  file = flog)
            print (_prodt_.qnt[RNUMBER], file = flog)

    # If at the previous iteration the supply was not completely sold, then the volume 
    # of the current supply is halved
    
    srcond = (_prodt_.nsl > 0.0)
    _prodt_.loc[srcond, 'qnt'] = _prodt_.loc[srcond, 'qnt']/2.0 

    # If the previous shipment was sold two times faster than the threshold time, 
    # then volume of current shipment is doubled. New threshold time (see below) will be equal to 
    # duration (the total sale time) of current shipment. 

    srcond = (_prodt_.stop/_prodt_.drn >= 2.0)
    _prodt_.loc[srcond, 'qnt'] = _prodt_.loc[srcond, 'qnt']*2.0
    _prodt_.loc[srcond, 'stop'] = 0
    _prodt_.loc(axis = 1)['qnt'] = np.floor (_prodt_.loc(axis = 1)['qnt']) 

    # Calculate slop of demand curves
    # Broadcasting and vectorization are used to minimise time of calculation

    k[...] = np.where (term > 10, 1.45/(0.2*term), 0.55/(0.5*term))

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('k:', file = flog)
            print (k[RNUMBER], file = flog)

    # Calculate TSFs and adding noise 

    sales[...] = (np.maximum (-np.tanh (k*(sales.T - term)), 0)*rng.normal(1, 0.25, size = (MAXTERM, _prodt_.shape[0]))).T

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('\nFirst sales:', file = flog)
            print (sales[RNUMBER], file = flog)

    # Form intervals of TDFs which correspond to intervals of shipments sales
        
    lts = [trends[lt][istart : istart + MAXTERM] for istart, lt in zip(_prodt_['start'], _prodt_['lt'])]
    
    alts = np.asarray (lts)
    
    # Calculate sale charts 

    sales[...] = ((sales*alts).T*ampli).T
    sales[...] = sales.clip (min = 0)

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('\nSales after alts:', file = flog)
            print (sales[RNUMBER], file = flog)

    if count == 0:
    # Calculate shipment volumes of the first shipments
        _prodt_.qnt = np.sum (sales, axis = 1)*rng.uniform (0.5, 0.8, size = _prodt_.shape[0])
        if RNUMBER > 0:
            with open (fpLog, 'a') as flog:
                print ('_prodt_.qnt:', file = flog)
                print (_prodt_.qnt[RNUMBER], file = flog)

    # Cut sales charts by threshold volume (_prodt_.qnt)
    # Note: since cumulative chart values may be not equal 
    # to threshold volume it is needed later to set the last value of chart cutted 
    # equal to _prodt_.qnt overwise values of _prodt_.qnt will decrease every itaration

    sales[...] = np.cumsum (sales, axis = 1)

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('\nSales after cumsum:', file = flog)
            print (sales[RNUMBER], file = flog)

    save[...] = sales.copy()

    sales *= (sales.T <= _prodt_.qnt.to_numpy()).T

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('\nSales after _prodt_.qnt:', file = flog)
            print (sales[RNUMBER], file = flog)

    sales[...] = np.where (sales.max (axis = 1) == 0.0, 0.0001, sales.T).T

    # Get duration of sale as time when cumulative chart achieve maximum value

    sales_max = sales.max (axis = 1)

    _prodt_.drn = np.array ([(np.isclose(x, sales_max[ind])).nonzero()[0].min() + 1 for ind, x in enumerate (list (sales))])
    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('_prodt_.drn: ', _prodt_.drn[RNUMBER], file = flog)

    sales[...] = np.where (sales_max < 1, sales.T, np.floor (sales.T)).T
    sales_max = sales.max (axis = 1)

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('\nSales after floor:', file = flog)
            print (sales[RNUMBER], file = flog)
            print ('\nSales_max:', file = flog)
            print (sales_max[RNUMBER], file = flog)

    if count == 0:
    #   Assign adjusted values to _prodt_.qnt after rounding sales charts
        _prodt_.loc(axis = 1)['qnt']  = sales_max
    #   The first value of threshold time is duration of the first shipment
        _prodt_.loc(axis = 1)['stop'] = _prodt_.drn

    if RNUMBER > 0:
        with open (fpLog, 'a') as flog:
            print ('_prodt_.qnt:', file = flog)
            print (_prodt_.qnt[RNUMBER], file = flog)

    # If duration of shipment sale is less then expiration period then 
    # the last value of cumulative sales chart is getting equal to _prodt_.qnt
    # This assignmeht allows to avoid gradual decrease of shipment volume.

    srcond = (_prodt_.drn < term)
    if RNUMBER > 0 and RNUMBER in _prodt_[srcond].index :
        with open (fpLog, 'a') as flog:
            print ('\nWE ARE BEFORE BASE SECTION', file = flog)
            print ('srcond:', file = flog)
            print (srcond, file = flog)    
    ixmask = np.ix_(srcond.to_numpy(), )
    if RNUMBER > 0 and RNUMBER in _prodt_[srcond].index :
        with open (fpLog, 'a') as flog:
            print ('ixmask:', file = flog)
            print (ixmask, file = flog)
    if ixmask[0].size > 0 and count != 0:
        smask = sales[ixmask]
        for ind, x in enumerate (list (_prodt_.drn.to_numpy()[ixmask])):
            smask[ind, x - 1] = _prodt_.loc[srcond, 'qnt'].to_numpy()[ind]
        sales[ixmask] = smask
        sales_max[ixmask] = smask.max (axis = 1)
        _prodt_.loc[srcond, 'nsl'] = 0
        _prodt_.loc[_prodt_.stop == 0, 'stop'] = _prodt_.drn
        if RNUMBER > 0 and RNUMBER in _prodt_[srcond].index :
            with open (fpLog, 'a') as flog:
                print ('\nWE ARE IN BASE SECTION', file = flog)
                print ('_prodt_.qnt:', file = flog)
                print (_prodt_.qnt[RNUMBER], file = flog)

    if RNUMBER > 0 :
        with open (fpLog, 'a') as flog:
            print ('\nSales after qnt correction:', file = flog)
            print (sales[RNUMBER], file = flog)

    # If shipment is not saled completly than threshold time is setting to expiration period 
    # _prodt_.nsl gets value of not saled volume and 
    # shipment volume becomes equals to volume soled

    srcond = (_prodt_.drn == term)
    ixmask = np.ix_(srcond.to_numpy(), )
    _prodt_.loc[srcond, 'stop'] = _prodt_[srcond].drn
    _prodt_.loc[srcond, 'nsl'] = (_prodt_[srcond].qnt - sales_max[ixmask]).clip (lower = 0)
    _prodt_.loc[srcond, 'qnt'] = sales_max[ixmask]
    
    if RNUMBER > 0 :
        with open (fpLog, 'a') as flog:
            print ('\nAfter nsl update:', file = flog)
            print ('_prodt_.stop: ', _prodt_.stop[RNUMBER], file = flog)
            print ('_prodt_.nsl:  ', _prodt_.nsl[RNUMBER],  file = flog)
            print ('_prodt_.qnt:  ', _prodt_.qnt[RNUMBER],  file = flog)

    # Percentile calculation.  
    # For ease of calculation, I use splines. To get percentiles, you need to have 
    # a function inverse to CDF. If there are periods on the cumulative sales chart where 
    # there are no sales, the inverse function is ambiguous.  
    # To get around this limitation, I add monotonically increasing values ​​<< 1 
    # to the sales graph at the points of no sales.

    sales[...] = (sales.T/sales.max(axis = 1)).T
 
    na_diff = na_diff_[:,1:]
    na_diff[np.isclose(np.diff (sales0, n = 1, axis = -1), 0.0).nonzero()] = 1
    na_diff[...] = np.cumsum (na_diff, -1)*na_diff/10000.0

    _spln_ = []
    
    if RNUMBER > 0 :
        with open (fpLog, 'a') as flog:
            print ('\nSales0 before splines:', file = flog)
            print (sales0[RNUMBER], file = flog)
            print ('\nna_diff before splines:', file = flog)
            print (na_diff[RNUMBER], file = flog)
            print ('\nnadrn before splines:', file = flog)
            print (_prodt_.drn[RNUMBER], file = flog)

    try:
        for ind, x in enumerate (list (sales0)):
            _slice_ = slice (0, _prodt_.drn[ind] + 1)
            x[_slice_] += na_diff_[ind, _slice_]
            fCDF = sp.interpolate.splrep (x[_slice_], np.arange(_prodt_.drn[ind] + 1), k=1)
            ptil = np.arange (0, 1.0, 0.049999999)
            sCDF = sp.interpolate.splev (ptil, fCDF)
            _spln_.append (sCDF)

    except Exception as e:
        print ('ind = ' + str(ind))
        print ('_prodt_.drn[ind] = ' + str(_prodt_.drn[ind]))
        print (x[slice(0, _prodt_.drn[ind])])
        print ('count = ', count)
        raise e

    dfCDF = pd.DataFrame (_spln_, columns = ['T00', 'T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20'])

    if count == 0:
        dfData = pd.merge (_prodt_, dfCDF, left_index=True, right_index=True)
    else:
        dfEpoch = pd.merge (_prodt_, dfCDF, left_index=True, right_index=True)
        dfData = pd.concat ([dfData, dfEpoch], axis = 0, ignore_index = True)

    _prodt_.drop (_prodt_[(_prodt_.start + _prodt_['drn']) > ihdays].index, inplace = True)
    _prodt_.reset_index (drop = True, inplace = True)

    lIND = _prodt_.index[_prodt_.phash == HASHID].tolist()
    RNUMBER = _prodt_.index.get_loc (lIND[0]) if len(lIND) > 0 else -100

    count += 1

with open (fpLog, 'a') as flog:
    print ('\ndfData.shape befoe drop: ', dfData.shape, file = flog) 

dfData.drop (dfData[dfData['qnt'] < 1].index, inplace = True)
dfData.reset_index (drop = True, inplace = True)

with open (fpLog, 'a') as flog:
    print ('\ndfData.shape after drop: ', dfData.shape, file = flog) 

# Create reference table dwf for words in descriptions

lwords = dfData['supplier'].unique().tolist() 
lwords.extend (dfData['product'].unique().tolist())
lwords.extend (dfData['nvalue' ].unique().tolist())
lwords.extend (dfData['volume' ].unique().tolist())
lwords.extend (dfData['packing'].unique().tolist())
lwords.extend (dfData['content'].unique().tolist())

lwords = [y for x in lwords for y in x.split()]
dfw = pd.DataFrame (list (set (lwords)), columns =['word'])
id  = np.arange (1, dfw.shape[0] + 1, dtype=int)
rng.shuffle (id)
dfw['wid'] = id
filepath = Path('ref_words.html')  
dfw.sort_values(['wid']).to_html (filepath, index = False)

# To distort the descriptions 3 variants of the word order are introduced and 
# reassigned randomly to shipments

dfData.insert (0, 'txtv', rng.integers (low = 1, high = 4, size = dfData.shape[0]))

conditions = [(dfData['txtv'] == 1), (dfData['txtv'] == 2), (dfData['txtv'] == 3)]

values = [
    dfData['supplier'] + ' ' + dfData['product'] + ' ' +
    dfData['nvalue']   + ' ' + dfData['volume']  + ' ' + 
    dfData['packing']  + ' ' + dfData['content'],
    dfData['supplier'] + ' ' + dfData['packing'] + ' ' +
    dfData['volume']   + ' ' + dfData['product'] + ' ' + 
    dfData['nvalue']   + ' ' + dfData['content'],
    dfData['product']  + ' ' + dfData['supplier']+ ' ' +
    dfData['volume']   + ' ' + dfData['nvalue']  + ' ' + 
    dfData['content']  + ' ' + dfData['packing']
    ]

dfData.insert (0, 'txt', np.select (conditions, values))
dfData['txt'] = dfData['txt'].replace ({'NA': ''}, regex = True)

# Split text resulted into words and encode each word using the referense table dfw
#dfText = dfData['txt'].str.split (expand = True)
#dfText.columns = ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6']

#for i, x in enumerate (dfText.columns):
#    dfw.columns = [x, 'w' + str(i)]
#    dfText = dfText.merge (dfw, how = 'left', left_on = x, right_on = x)

#dfData = dfText.merge (dfData, left_index = True, right_index = True)

# Convert shipment date into datetime64 format
nastart = np.asarray([d64start + np.timedelta64 (i, 'D') for i in dfData['start']])

dfData.insert (0, 'is', dfData['start'])
dfData.drop (['start'], axis = 1, inplace = True)
dfData.insert (0, 'start', nastart)

# Create derivatives of shipment date
dfData.insert (1, 'yy', dfData.start.dt.year)
dfData.insert (2, 'mm', dfData.start.dt.month)
dfData.insert (3, 'dd', dfData.start.dt.day)
dfData.insert (4, 'wd', dfData.start.dt.dayofweek)

# Create reference tables for product groups, suppliers, outbound locations, clients

dfpg = pd.DataFrame (dfData['product'].unique(), columns =['product'])
id = np.arange (1, dfpg.shape[0] + 1, dtype=int)
rng.shuffle (id)
dfpg['gid'] = id
dfData = dfpg.merge (dfData, how = 'inner', left_on ='product', right_on ='product')
filepath = Path('ref_groups.html')  
dfpg.sort_values(['gid']).to_html (filepath, index = False)

dfsp = pd.DataFrame (dfData['supplier'].unique(), columns =['supplier'])
id = np.arange (1, dfsp.shape[0] + 1, dtype=int)
rng.shuffle (id)
dfsp['sid'] = id
dfData = dfsp.merge (dfData, how = 'inner', left_on ='supplier', right_on ='supplier')
filepath = Path('ref_suppliers.html')  
dfsp.sort_values(['sid']).to_html (filepath, index = False)

dfol = pd.DataFrame (dfData['oloc'].unique(), columns =['oloc'])
id = np.arange (1, dfol.shape[0] + 1, dtype=int)
rng.shuffle (id)
dfol['oid'] = id
dfData = dfol.merge (dfData, how = 'inner', left_on ='oloc', right_on ='oloc')
filepath = Path('ref_olos.html')  
dfol.sort_values(['oid']).to_html (filepath, index = False)

dfcl = pd.DataFrame (dfData['client'].unique(), columns =['client'])
id = np.arange (1, dfcl.shape[0] + 1, dtype=int)
rng.shuffle (id)
dfcl['cid'] = id
dfData = dfcl.merge (dfData, how = 'inner', on ='client')
dfData = clnt.merge (dfData, how = 'inner', on ='client')
filepath = Path('ref_clients.html')  
dfcl.sort_values(['cid']).to_html (filepath, index = False)

fphml = Path ('inidata.html')
lprint = [
    'start','yy','mm','dd','client','oloc','supplier',
    'product','nvalue','volume','packing','content',
    'term','lt','nsl','drn','qnt','stop','phash',
    'T00','T01','T02','T03','T04','T05'
    ]
dfData.loc[dfData.phash == HASHID, lprint].sort_values (by = lhash + ['start']).to_html (fphml, float_format = '{:,.2f}'.format, index=True)

dfData = dfData.merge (dfdesc, how = 'inner', on = ['supplier', 'product', 'nvalue', 'volume', 'packing', 'content'])

#ldrop = [
#    'supplier', 'product', 'nvalue', 'volume', 
#    'packing', 'content', 'txtv', 'txt', 'start',
#    'ampli', 'lt', 'term', 'client', 'oloc', 'gid',
#    'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'T00'
#    ]
ldrop = [
    'supplier', 'product', 'nvalue', 'volume', 
    'packing', 'content', 'txtv', 'start', 'ampli', 
    'lt', 'term', 'client', 'oloc', 'gid', 'T00'
    ]
dfData.drop (ldrop, axis=1, inplace = True)

dfData = dfData.sample (frac = 1).reset_index (drop = True)

# Normalize fields of _lX_ list (see below) but before save integer values 
# for 'group by' operations and testing

dfData.rename (columns = {'is': 'is_i'}, inplace = True)
dfData['tid_i'] = dfData['tid'].astype(np.int32)
dfData['cid_i'] = dfData['cid'].astype(np.int32)
dfData['oid_i'] = dfData['oid'].astype(np.int32)
dfData['sid_i'] = dfData['sid'].astype(np.int32)
dfData['did_i'] = dfData['did'].astype(np.int32)

_lX_ = ['tid', 'cid', 'oid', 'sid', 'did', 'yy', 'mm', 'dd', 'wd', 'qnt', 'stop']

dfX = dfData.loc(axis = 1)[_lX_]
dfX_stats = dfX.describe().transpose()
dfData.loc(axis = 1)[_lX_] = (dfX - dfX_stats['mean'])/dfX_stats['std']

fpcsv = Path ('inidata.csv')  
dfData.to_csv (fpcsv, index = False)
